In [ ]:
!pip install pyspark
!pip install -q findspark
!pip install pyspark-stubs
#!pip install pyfpgrowth
#!pip3 install mlxtend --user
#%pip install mlxtend --upgrade
!pip install -q findspark
#!pip install sklearn

In [ ]:
import pandas as pd
import collections, numpy as np
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline

In [ ]:
from pyspark.ml.fpm import FPGrowth
from pyspark.ml.fpm import PrefixSpan
# $example off$
from pyspark.sql import SparkSession,Row
from pyspark.sql.types import *
from pyspark.sql.functions import collect_set, col, count,explode,collect_list
#import pyspark.sql.functions as F

In [ ]:
spark = SparkSession\
.builder\
.appName("FPGrowthExample")\
.getOrCreate()

In [ ]:
allergy = "../input/healthcare-dataset/allergies.csv"
careplans = "../input/healthcare-dataset/careplans.csv"
claims = "../input/healthcare-dataset/claims.csv"
devices = "../input/healthcare-dataset/devices.csv"
encounters = "../input/healthcare-dataset/encounters.csv"
imaging_studies = "../input/healthcare-dataset/imaging_studies.csv"
immunuzation = "../input/healthcare-dataset/immunizations.csv"
medication = "../input/healthcare-dataset/medications.csv"
observation = "../input/healthcare-dataset/observations.csv"
organization = "../input/healthcare-dataset/organizations.csv"
patient = "../input/healthcare-dataset/patients.csv"
payer_transaction = "../input/healthcare-dataset/payer_transitions.csv"
payers = "../input/healthcare-dataset/payers.csv"
procedure = "../input/healthcare-dataset/procedures.csv"
providers = "../input/healthcare-dataset/providers.csv"
supplies = "../input/healthcare-dataset/supplies.csv"
claim_transaction = "../input/healthcare-dataset/claims_transactions.csv"
condition = "../input/healthcare-dataset/conditions.csv"


In [ ]:
pat1 = (spark.read.format("csv").options(header="true").option("inferSchema","true").load("../input/healthcare-dataset/patients.csv"))
#pat1 = spark.read.csv("/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/patients.csv", header=True, inferSchema=True)
pat1.createOrReplaceTempView("patient")
pat1.printSchema()
#pat1.columns

In [ ]:
enco1 = (spark.read.format("csv").options(header="true").option("inferSchema","true").load("../input/healthcare-dataset/encounters.csv"))
#pat1 = spark.read.csv("/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/patients.csv", header=True, inferSchema=True)
enco1.createOrReplaceTempView("encounter")
print(enco1.printSchema())
print(enco1.show())
#pat1.columns

In [ ]:
#enco_pat = pd.merge(enco, pat, left_on="PATIENT", right_on="Id")
enco1_pat1 = enco1.join(pat1,enco1.PATIENT ==  pat1.Id, "inner")
enco1_pat1.show(30)
enco1_pat1.count()

In [ ]:
#rawData = spark.sql("select p.product_name, o.order_id from products p inner join order_products_train o where o.product_id = p.product_id")
pat_data = spark.sql("select p.Id, e.ENCOUNTERCLASS,e.TOTAL_CLAIM_COST from encounter e , patient p where p.Id == e.PATIENT")
#print(pat_data)
print(pat_data.schema.names)
#print(pat_data.printSchema())
#pat_class_schema = pat_data.groupBy('Id').agg(collect_set('ENCOUNTERCLASS').alias('items'))
print(pat_data.show())
pat_data.count()

In [ ]:
pat1_class1 = pat_data.groupBy('Id').agg(collect_list("TOTAL_CLAIM_COST").alias('items1'),collect_list("ENCOUNTERCLASS").alias('items2'))
pat1_class1.createOrReplaceTempView('class1')
print(pat1_class1.count())
pat1_class1.show()

In [ ]:
pat_dataDF = pat_data.select("*").toPandas()
print(pat_dataDF['TOTAL_CLAIM_COST'].isna().sum() )
class1DF = pat1_class1.select("*").toPandas()
print(class1DF['items1'].isna().sum() )

In [ ]:
pat1_class2 = pat_data.groupBy('Id').agg(collect_set('ENCOUNTERCLASS').alias('items'))
pat1_class2.createOrReplaceTempView('class2')
print(pat1_class2.count())
pat1_class2.show()

In [ ]:
pat1_class3 = pat_data.groupBy('Id').agg(collect_set('TOTAL_CLAIM_COST').alias('items'))
pat1_class3.createOrReplaceTempView('class3')
print(pat1_class3.count())
pat1_class3.show()

In [ ]:
#%scala
%time
from pyspark.ml.fpm import FPGrowth

# Extract out the items 
class1_itemsDF = spark.sql("select items from class3")#.as[Array[String]].toDF("items")

################# IT IS SHOWING PREDICTED DATA OF TOTAL_CLAIM_COST

# Use FPGrowth
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.006, minConfidence=0.006)
model = fpGrowth.fit(class1_itemsDF)
# Display frequent itemsets.
model.freqItemsets.show()
mostPopularPatClass = model.freqItemsets

model.associationRules
rules_TClaim_cost = model.associationRules
rules_TClaim_cost.show()  
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(class1_itemsDF).show()
rules_TClaim_cost.createOrReplaceTempView("rules_TClaim_cost_table")

In [ ]:
#%scala
%time
from pyspark.ml.fpm import FPGrowth

####################### IT IS SHOWING PREDICTED DATA OF ENCOUNTERCLASS

# Extract out the items 
class1_itemsDF = spark.sql("select items from class2")#.as[Array[String]].toDF("items")

# Use FPGrowth
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.006, minConfidence=0.006)
model = fpGrowth.fit(class1_itemsDF)
# Display frequent itemsets.
model.freqItemsets.show()
mostPopularPatClass = model.freqItemsets

model.associationRules
rules_data = model.associationRules
rules_data.show()  
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(class1_itemsDF)
#rule.createOrReplaceTempView("rule_table")

In [ ]:
rulesDF = rules_data.select("*").toPandas()
rulesDF.head()
rulesDF.shape

In [ ]:
rulesDF[rulesDF["antecedent"].apply(lambda x: "urgentcare" in str(x))].sort_values(ascending=False,by='lift')

In [ ]:
rulesTC_cost = rules_TClaim_cost.select("*").toPandas()
rulesTC_cost.head()
rulesTC_cost.shape

In [ ]:
rulesTC_cost[rulesTC_cost["antecedent"].apply(lambda x: "926.85" in str(x))].sort_values(ascending=False,by='lift')

In [ ]:
dataframe = pd.concat([rulesTC_cost, rulesDF], axis=0)


In [ ]:
dataframe.iloc[15398,:]

**Start of merging DataFrames******

In [ ]:
allergy = pd.read_csv(allergy)                    ## small dataset
careplans = pd.read_csv(careplans)                ## medium datset
claims = pd.read_csv(claims)                      ## large dataset
devices = pd.read_csv(devices)                   ## small dataset
encounters = pd.read_csv(encounters)              ## medium dataset
imaging_studies = pd.read_csv(imaging_studies)   ## large dataset
immunuzation = pd.read_csv(immunuzation)         ## small dataset
medication = pd.read_csv(medication)             ## large dataset
observation = pd.read_csv(observation)           ## very large dataset
organization = pd.read_csv(organization)         ## small dataset
patient = pd.read_csv(patient)                    ## small dataset
payer_transaction = pd.read_csv(payer_transaction) ## large dataset
payers = pd.read_csv(payers)                     ## small dataset
procedure = pd.read_csv(procedure)               ## large dataset
providers = pd.read_csv(providers)                ## medium dataset 
supplies = pd.read_csv(supplies)                 ## large dataset
claim_transaction = pd.read_csv(claim_transaction)  ## very large dataset
condition = pd.read_csv(condition)               ## medium dataset

In [ ]:
allergy.head()
print(encounters.head())
#patinet = patinet.select("*").toPandas()
claims.head()

In [ ]:
alg_pat = pd.merge(allergy, patient, left_on="PATIENT", right_on="Id")
print(alg_pat.columns)
print(len(alg_pat['Id'].unique())," | ",alg_pat.shape)

In [ ]:
enco = encounters.head(5000)
enco_alg_pat = pd.merge(enco, alg_pat, left_on="PATIENT", right_on="Id")
print(enco_alg_pat.columns)
print(len(enco_alg_pat['Id_y'].unique())," | ",enco_alg_pat.shape)

In [ ]:
#this merging is giving zero record because there is no primary and foreign key is matching, so we will merge this later
#enco_alg_pat_prov_carpln_claim_dev = pd.merge(enco_alg_pat_prov_carpln_claim,devices, left_on="Id_y",right_on="PATIENT")
#print(enco_alg_pat_prov_carpln_claim_dev.columns)

In [ ]:
prov = providers.head(5000)
enco_alg_pat_prov = pd.merge(enco_alg_pat,prov, left_on="PROVIDER", right_on="Id")
enco_alg_pat_prov.columns

In [ ]:
print(len(enco_alg_pat_prov['Id_y'].unique())," | ",enco_alg_pat_prov.shape)

In [ ]:
carpln = careplans.head(5000)
enco_alg_pat_prov_carpln = pd.merge(enco_alg_pat_prov,carpln, left_on="Id_y",right_on="PATIENT")
print(enco_alg_pat_prov_carpln.columns)

In [ ]:
df_columns = enco_alg_pat_prov_carpln.columns.values.tolist()
print(df_columns)
column_dup = [k for k,v in Counter(df_columns).items() if v > 1]
print(column_dup)

In [ ]:
enco_alg_pat_prov_carpln.set_axis(['Id_w', 'START_x', 'STOP_x', 'PATIENT_x', 'ORGANIZATION_x', 'PROVIDER',
                                   'PAYER', 'ENCOUNTERCLASS', 'CODE_x', 'DESCRIPTION_x',
                                   'BASE_ENCOUNTER_COST', 'TOTAL_CLAIM_COST', 'PAYER_COVERAGE',
                                   'REASONCODE_x', 'REASONDESCRIPTION_x', 'START_y', 'STOP_y', 'PATIENT_y',
                                   'ENCOUNTER_x', 'CODE_y', 'SYSTEM', 'DESCRIPTION_y', 'TYPE', 'CATEGORY',
                                   'REACTION1', 'DESCRIPTION1', 'SEVERITY1', 'REACTION2', 'DESCRIPTION2',
                                   'SEVERITY2', 'Id_y', 'BIRTHDATE', 'DEATHDATE', 'SSN', 'DRIVERS',
                                   'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL',
                                   'RACE', 'ETHNICITY', 'GENDER_x', 'BIRTHPLACE', 'ADDRESS_x', 'CITY_x',
                                   'STATE_x', 'COUNTY', 'ZIP_x', 'LAT_x', 'LON_x', 'HEALTHCARE_EXPENSES',
                                   'HEALTHCARE_COVERAGE', 'Id_x', 'ORGANIZATION_y', 'NAME', 'GENDER_y',
                                   'SPECIALITY', 'ADDRESS_y', 'CITY_y', 'STATE_y', 'ZIP_y', 'LAT_y',
                                   'LON_y', 'UTILIZATION', 'Id_z', 'START', 'STOP', 'PATIENT',
                                   'ENCOUNTER_y', 'CODE', 'DESCRIPTION', 'REASONCODE_y','REASONDESCRIPTION_y'], axis=1, inplace=True)
enco_alg_pat_prov_carpln.columns

In [ ]:
print(enco_alg_pat_prov_carpln.columns)

In [ ]:
print(len(enco_alg_pat_prov_carpln['Id_y'].unique())," | ",enco_alg_pat_prov_carpln.shape)

In [ ]:
clam = claims.head(2300)
clam.shape

In [ ]:
enco_alg_pat_prov_carpln_claim = pd.merge(enco_alg_pat_prov_carpln,clam, left_on="Id_y",right_on="PATIENTID")
print(enco_alg_pat_prov_carpln_claim.columns)

In [ ]:
df_columns = enco_alg_pat_prov_carpln_claim.columns.values.tolist()
print(df_columns)
column_dup = [k for k,v in Counter(df_columns).items() if v > 1]
print(column_dup)

In [ ]:
print(len(enco_alg_pat_prov_carpln_claim['Id_y'].unique())," | ",enco_alg_pat_prov_carpln_claim.shape)

In [ ]:
#this merging is giving zero record because there is no primary and foreign key is matching, so we will merge this later
#enco_alg_pat_prov_carpln_claim_dev = pd.merge(enco_alg_pat_prov_carpln_claim,devices, left_on="Id_y",right_on="PATIENT")
#print(enco_alg_pat_prov_carpln_claim_dev.columns)

In [ ]:
immun = immunuzation.head(500)
enco_alg_pat_prov_carpln_claim_immun = pd.merge(enco_alg_pat_prov_carpln_claim,immun, left_on="Id_y",right_on="PATIENT")
print(enco_alg_pat_prov_carpln_claim_immun.columns)

In [ ]:
print(len(enco_alg_pat_prov_carpln_claim_immun['Id_y'].unique())," | ",enco_alg_pat_prov_carpln_claim_immun.shape)

In [ ]:
df_columns = enco_alg_pat_prov_carpln_claim_immun.columns.values.tolist()
print(df_columns)
column_dup = [k for k,v in Counter(df_columns).items() if v > 1]
print(column_dup)

In [ ]:
columns_name = ['Id_w', 'START_x', 'STOP_x', 'PATIENT_x', 'ORGANIZATION_x', 'PROVIDER', 'PAYER', 'ENCOUNTERCLASS', 'CODE_x', 'DESCRIPTION_x', 'BASE_ENCOUNTER_COST','TOTAL_CLAIM_COST', 'PAYER_COVERAGE', 'REASONCODE_x', 'REASONDESCRIPTION_x', 'START_y', 'STOP_y', 'PATIENT_y', 'ENCOUNTER_x', 'CODE_y', 'SYSTEM','DESCRIPTION_y', 'TYPE', 'CATEGORY', 'REACTION1', 'DESCRIPTION1', 'SEVERITY1','REACTION2', 'DESCRIPTION2', 'SEVERITY2', 'Id_y', 'BIRTHDATE', 'DEATHDATE','SSN', 'DRIVERS', 'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN','MARITAL', 'RACE', 'ETHNICITY', 'GENDER_x', 'BIRTHPLACE', 'ADDRESS_x', 'CITY_x','STATE_x', 'COUNTY', 'ZIP_x', 'LAT_x', 'LON_x', 'HEALTHCARE_EXPENSES','HEALTHCARE_COVERAGE', 'Id_x', 'ORGANIZATION_y', 'NAME', 'GENDER_y','SPECIALITY', 'ADDRESS_y', 'CITY_y', 'STATE_y', 'ZIP_y', 'LAT_y', 'LON_y','UTILIZATION', 'Id_z', 'START', 'STOP', 'PATIENT_careplan', 'ENCOUNTER_y', 'CODE_careplan','DESCRIPTION_careplan', 'REASONCODE_y', 'REASONDESCRIPTION_y', 'Id', 'PATIENTID','PROVIDERID', 'PRIMARYPATIENTINSURANCEID', 'SECONDARYPATIENTINSURANCEID','DEPARTMENTID', 'PATIENTDEPARTMENTID', 'DIAGNOSIS1', 'DIAGNOSIS2', 'DIAGNOSIS3','DIAGNOSIS4', 'DIAGNOSIS5', 'DIAGNOSIS6', 'DIAGNOSIS7', 'DIAGNOSIS8','REFERRINGPROVIDERID', 'APPOINTMENTID', 'CURRENTILLNESSDATE', 'SERVICEDATE','SUPERVISINGPROVIDERID', 'STATUS1', 'STATUS2', 'STATUSP', 'OUTSTANDING1','OUTSTANDING2', 'OUTSTANDINGP', 'LASTBILLEDDATE1', 'LASTBILLEDDATE2','LASTBILLEDDATEP', 'HEALTHCARECLAIMTYPEID1', 'HEALTHCARECLAIMTYPEID2', 'DATE','PATIENT_immunization', 'ENCOUNTER', 'CODE_immunization', 'DESCRIPTION_immunization', 'BASE_COST']
enco_alg_pat_prov_carpln_claim_immun.set_axis(columns_name, axis=1, inplace=True)
enco_alg_pat_prov_carpln_claim_immun.columns

In [ ]:
#With 1000 rows it is giving this df.shape = (4374240, 126), so I am using 560 rows
med = medication.head(560)
enco_alg_pat_prov_carpln_claim_immun_med = pd.merge(enco_alg_pat_prov_carpln_claim_immun,med, left_on="Id_y",right_on="PATIENT")
print(enco_alg_pat_prov_carpln_claim_immun_med.columns)

In [ ]:
print(len(enco_alg_pat_prov_carpln_claim_immun_med['Id_y'].unique())," | ",enco_alg_pat_prov_carpln_claim_immun_med.shape)

In [ ]:
df_columns = enco_alg_pat_prov_carpln_claim_immun.columns.values.tolist()
print(df_columns)
column_dup = [k for k,v in Counter(df_columns).items() if v > 1]
print(column_dup)

In [ ]:
#It is taking alot of memory merging organization data so I am using first 300 rows of organization dataset
org = organization.head(300)
enco_alg_pat_prov_carpln_claim_immun_med_org = pd.merge(enco_alg_pat_prov_carpln_claim_immun_med,org, left_on="ORGANIZATION_x",right_on="Id")
print(enco_alg_pat_prov_carpln_claim_immun_med_org.columns)

In [ ]:
df_columns = enco_alg_pat_prov_carpln_claim_immun_med_org.columns.values.tolist()
#print(df_columns)
column_dup = [k for k,v in Counter(df_columns).items() if v > 1]
print(column_dup)

In [ ]:
columns_name = ['Id_w', 'START_alg', 'STOP_alg', 'PATIENT_x', 'ORGANIZATION_x', 'PROVIDER', 'PAYER_x', 
                'ENCOUNTERCLASS', 'CODE_x', 'DESCRIPTION_x', 'BASE_ENCOUNTER_COST', 'TOTAL_CLAIM_COST',
                'PAYER_COVERAGE_x', 'REASONCODE_x', 'REASONDESCRIPTION_x', 'START_enco', 'STOP_enco', 'PATIENT_y',
                'ENCOUNTER_enco', 'CODE_y', 'SYSTEM', 'DESCRIPTION_y', 'TYPE', 'CATEGORY', 'REACTION1', 
                'DESCRIPTION1', 'SEVERITY1', 'REACTION2', 'DESCRIPTION2', 'SEVERITY2', 'Id_pat', 'BIRTHDATE',
                'DEATHDATE', 'SSN', 'DRIVERS', 'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN',
                'MARITAL', 'RACE', 'ETHNICITY', 'GENDER_x', 'BIRTHPLACE', 'ADDRESS_x', 'CITY_x', 'STATE_x',
                'COUNTY', 'ZIP_x', 'LAT_x', 'LON_x', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE', 'Id_prov',
                'ORGANIZATION_y', 'NAME_x', 'GENDER_y', 'SPECIALITY', 'ADDRESS_y', 'CITY_y', 'STATE_y',
                'ZIP_y', 'LAT_y', 'LON_y', 'UTILIZATION_x', 'Id_careplan', 'START_careplan', 'STOP_careplan', 'PATIENT_careplan',
                'ENCOUNTER_careplan', 'CODE_careplan', 'DESCRIPTION_careplan', 'REASONCODE_y', 'REASONDESCRIPTION_y',
                'Id_x', 'PATIENTID', 'PROVIDERID', 'PRIMARYPATIENTINSURANCEID', 'SECONDARYPATIENTINSURANCEID',
                'DEPARTMENTID', 'PATIENTDEPARTMENTID', 'DIAGNOSIS1', 'DIAGNOSIS2', 'DIAGNOSIS3', 'DIAGNOSIS4',
                'DIAGNOSIS5', 'DIAGNOSIS6', 'DIAGNOSIS7', 'DIAGNOSIS8', 'REFERRINGPROVIDERID', 'APPOINTMENTID',
                'CURRENTILLNESSDATE', 'SERVICEDATE', 'SUPERVISINGPROVIDERID', 'STATUS1', 'STATUS2', 'STATUSP',
                'OUTSTANDING1', 'OUTSTANDING2', 'OUTSTANDINGP', 'LASTBILLEDDATE1', 'LASTBILLEDDATE2',
                'LASTBILLEDDATEP', 'HEALTHCARECLAIMTYPEID1', 'HEALTHCARECLAIMTYPEID2', 'DATE',
                'PATIENT_immunization', 'ENCOUNTER_x', 'CODE_immunization', 'DESCRIPTION_immunization',
                'BASE_COST_x', 'START_y', 'STOP_y', 'PATIENT', 'PAYER_y', 'ENCOUNTER_y', 'CODE', 'DESCRIPTION',
                'BASE_COST_y', 'PAYER_COVERAGE_y', 'DISPENSES', 'TOTALCOST', 'REASONCODE', 'REASONDESCRIPTION',
                'Id_y', 'NAME_y', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'LAT', 'LON', 'PHONE', 'REVENUE', 
                'UTILIZATION_y']
enco_alg_pat_prov_carpln_claim_immun_med_org.set_axis(columns_name, axis=1, inplace=True)
#enco_alg_pat_prov_carpln_claim_immun.columns

In [ ]:
print(len(enco_alg_pat_prov_carpln_claim_immun_med_org['Id_pat'].unique())," | ",enco_alg_pat_prov_carpln_claim_immun_med_org.shape)

In [ ]:
pytran = payer_transaction.head(1400)
enco_alg_pat_prov_carpln_claim_immun_med_org_pytran = pd.merge(enco_alg_pat_prov_carpln_claim_immun_med_org, pytran, left_on="Id_pat",right_on="PATIENT")
print(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran.columns)

In [ ]:
df_columns = enco_alg_pat_prov_carpln_claim_immun_med_org_pytran.columns.values.tolist()
#print(df_columns)
column_dup = [k for k,v in Counter(df_columns).items() if v > 1]
print(column_dup)

In [ ]:
print(len(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran['Id_pat'].unique())," | ",enco_alg_pat_prov_carpln_claim_immun_med_org_pytran.shape)

In [ ]:
columns_name = ['Id_w', 'START_alg', 'STOP_alg', 'PATIENT_alg', 'ORGANIZATION_x', 'PROVIDER', 
                'PAYER_x', 'ENCOUNTERCLASS', 'CODE_x', 'DESCRIPTION_x', 'BASE_ENCOUNTER_COST', 
                'TOTAL_CLAIM_COST', 'PAYER_COVERAGE_x', 'REASONCODE_x', 'REASONDESCRIPTION_x', 
                'START_enco', 'STOP_enco', 'PATIENT_y', 'ENCOUNTER_enco', 'CODE_y', 'SYSTEM', 
                'DESCRIPTION_y', 'TYPE', 'CATEGORY', 'REACTION1', 'DESCRIPTION1', 'SEVERITY1',
                'REACTION2', 'DESCRIPTION2', 'SEVERITY2', 'Id_pat', 'BIRTHDATE', 'DEATHDATE', 'SSN', 
                'DRIVERS', 'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL', 'RACE',
                'ETHNICITY', 'GENDER_x', 'BIRTHPLACE', 'ADDRESS_x', 'CITY_x', 'STATE_x', 'COUNTY', 
                'ZIP_x', 'LAT_x', 'LON_x', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE', 'Id_prov', 
                'ORGANIZATION_y', 'NAME_x', 'GENDER_y', 'SPECIALITY', 'ADDRESS_y', 'CITY_y', 'STATE_y',
                'ZIP_y', 'LAT_y', 'LON_y', 'UTILIZATION_x', 'Id_careplan', 'START_careplan', 
                'STOP_careplan', 'PATIENT_careplan', 'ENCOUNTER_careplan', 'CODE_careplan', 
                'DESCRIPTION_careplan', 'REASONCODE_y', 'REASONDESCRIPTION_y', 'Id_x', 'PATIENTID',
                'PROVIDERID', 'PRIMARYPATIENTINSURANCEID', 'SECONDARYPATIENTINSURANCEID', 'DEPARTMENTID',
                'PATIENTDEPARTMENTID', 'DIAGNOSIS1', 'DIAGNOSIS2', 'DIAGNOSIS3', 'DIAGNOSIS4', 'DIAGNOSIS5',
                'DIAGNOSIS6', 'DIAGNOSIS7', 'DIAGNOSIS8', 'REFERRINGPROVIDERID', 'APPOINTMENTID', 
                'CURRENTILLNESSDATE', 'SERVICEDATE', 'SUPERVISINGPROVIDERID', 'STATUS1', 'STATUS2',
                'STATUSP', 'OUTSTANDING1', 'OUTSTANDING2', 'OUTSTANDINGP', 'LASTBILLEDDATE1',
                'LASTBILLEDDATE2', 'LASTBILLEDDATEP', 'HEALTHCARECLAIMTYPEID1', 'HEALTHCARECLAIMTYPEID2',
                'DATE', 'PATIENT_immunization', 'ENCOUNTER_x', 'CODE_immunization', 
                'DESCRIPTION_immunization', 'BASE_COST_x', 'START_y', 'STOP_y', 'PATIENT_med', 'PAYER_y',
                'ENCOUNTER_y', 'CODE', 'DESCRIPTION', 'BASE_COST_y', 'PAYER_COVERAGE_y', 'DISPENSES',
                'TOTALCOST', 'REASONCODE', 'REASONDESCRIPTION', 'Id_y', 'NAME_y', 'ADDRESS', 'CITY', 
                'STATE', 'ZIP', 'LAT', 'LON', 'PHONE', 'REVENUE', 'UTILIZATION_y', 'PATIENT_pytran', 'MEMBERID',
                'START_YEAR', 'END_YEAR', 'PAYER', 'SECONDARY_PAYER', 'OWNERSHIP', 'OWNERNAME']
enco_alg_pat_prov_carpln_claim_immun_med_org_pytran.set_axis(columns_name, axis=1, inplace=True)


In [ ]:
enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer = pd.merge(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran, payers, left_on="PAYER",right_on="Id")
print(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer.columns)

In [ ]:
df_columns = enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer.columns.values.tolist()
#print(df_columns)
column_dup = [k for k,v in Counter(df_columns).items() if v > 1]
print(column_dup)

In [ ]:
columns_name = ['Id_w', 'START_alg', 'STOP_alg', 'PATIENT_alg', 'ORGANIZATION_x', 'PROVIDER', 'PAYER_x', 'ENCOUNTERCLASS', 
                'CODE_x', 'DESCRIPTION_x', 'BASE_ENCOUNTER_COST', 'TOTAL_CLAIM_COST', 'PAYER_COVERAGE_x', 'REASONCODE_x', 
                'REASONDESCRIPTION_x', 'START_enco', 'STOP_enco', 'PATIENT_y', 'ENCOUNTER_enco', 'CODE_y', 'SYSTEM', 'DESCRIPTION_y',
                'TYPE', 'CATEGORY', 'REACTION1', 'DESCRIPTION1', 'SEVERITY1', 'REACTION2', 'DESCRIPTION2', 'SEVERITY2', 'Id_pat',
                'BIRTHDATE', 'DEATHDATE', 'SSN', 'DRIVERS', 'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL',
                'RACE', 'ETHNICITY', 'GENDER_x', 'BIRTHPLACE', 'ADDRESS_x', 'CITY_x', 'STATE_x', 'COUNTY', 'ZIP_x', 'LAT_x', 'LON_x',
                'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE', 'Id_prov', 'ORGANIZATION_y', 'NAME_x', 'GENDER_y', 'SPECIALITY', 
                'ADDRESS_y', 'CITY_y', 'STATE_y', 'ZIP_y', 'LAT_y', 'LON_y', 'UTILIZATION_x', 'Id_careplan', 'START_careplan', 
                'STOP_careplan', 'PATIENT_careplan', 'ENCOUNTER_careplan', 'CODE_careplan', 'DESCRIPTION_careplan', 'REASONCODE_y',
                'REASONDESCRIPTION_y', 'Id_x', 'PATIENTID', 'PROVIDERID', 'PRIMARYPATIENTINSURANCEID', 'SECONDARYPATIENTINSURANCEID',
                'DEPARTMENTID', 'PATIENTDEPARTMENTID', 'DIAGNOSIS1', 'DIAGNOSIS2', 'DIAGNOSIS3', 'DIAGNOSIS4', 'DIAGNOSIS5',
                'DIAGNOSIS6', 'DIAGNOSIS7', 'DIAGNOSIS8', 'REFERRINGPROVIDERID', 'APPOINTMENTID', 'CURRENTILLNESSDATE', 
                'SERVICEDATE', 'SUPERVISINGPROVIDERID', 'STATUS1', 'STATUS2', 'STATUSP', 'OUTSTANDING1', 'OUTSTANDING2',
                'OUTSTANDINGP', 'LASTBILLEDDATE1', 'LASTBILLEDDATE2', 'LASTBILLEDDATEP', 'HEALTHCARECLAIMTYPEID1', 
                'HEALTHCARECLAIMTYPEID2', 'DATE', 'PATIENT_immunization', 'ENCOUNTER_x', 'CODE_immunization', 
                'DESCRIPTION_immunization', 'BASE_COST_x', 'START_y', 'STOP_y', 'PATIENT_med', 'PAYER_y', 'ENCOUNTER_y', 'CODE',
                'DESCRIPTION', 'BASE_COST_y', 'PAYER_COVERAGE_y', 'DISPENSES', 'TOTALCOST', 'REASONCODE', 'REASONDESCRIPTION', 
                'Id_org', 'NAME_org', 'ADDRESS_org', 'CITY_org', 'STATE', 'ZIP_org', 'LAT', 'LON', 'PHONE_x', 'REVENUE_x', 'UTILIZATION_y', 
                'PATIENT_pytran', 'MEMBERID', 'START_YEAR', 'END_YEAR', 'PAYER', 'SECONDARY_PAYER', 'OWNERSHIP', 'OWNERNAME', 'Id',
                'NAME', 'ADDRESS_payer', 'CITY_payer', 'STATE_HEADQUARTERED', 'ZIP_payer', 'PHONE_y', 'AMOUNT_COVERED', 'AMOUNT_UNCOVERED', 
                'REVENUE_y', 'COVERED_ENCOUNTERS', 'UNCOVERED_ENCOUNTERS', 'COVERED_MEDICATIONS', 'UNCOVERED_MEDICATIONS',
                'COVERED_PROCEDURES', 'UNCOVERED_PROCEDURES', 'COVERED_IMMUNIZATIONS', 'UNCOVERED_IMMUNIZATIONS', 'UNIQUE_CUSTOMERS',
                'QOLS_AVG', 'MEMBER_MONTHS']
enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer.set_axis(columns_name, axis=1, inplace=True)

In [ ]:
#Payers dataset has not been added due to less foreign key data of Payers Dataset in merged dataset
print(len(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer['Id_pat'].unique())," | ",enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer.shape)

In [ ]:
#obsr = observation.head(9750)
#enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_obsr = pd.merge(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer, obsr, left_on="Id_pat",right_on="PATIENT")
#print(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_obsr.columns)

In [ ]:
#df_columns = enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_obsr.columns.values.tolist()
#print(df_columns)
#column_dup = [k for k,v in Counter(df_columns).items() if v > 1]
#print(column_dup)

In [ ]:
"""
columns_name = ['Id_w', 'START_enco', 'STOP_enco', 'PATIENT_enco', 'ORGANIZATION_x', 'PROVIDER', 'PAYER_x', 'ENCOUNTERCLASS', 'CODE_enco', 
                'DESCRIPTION_enco', 'BASE_ENCOUNTER_COST', 'TOTAL_CLAIM_COST', 'PAYER_COVERAGE_x', 'REASONCODE_x', 'REASONDESCRIPTION_x',
                'START_alg', 'STOP_alg', 'PATIENT_y', 'ENCOUNTER_alg', 'CODE_alg', 'SYSTEM', 'DESCRIPTION_alg', 'TYPE_x', 'CATEGORY_x',
                'REACTION1', 'DESCRIPTION1', 'SEVERITY1', 'REACTION2', 'DESCRIPTION2', 'SEVERITY2', 'Id_pat', 'BIRTHDATE', 'DEATHDATE',
                'SSN', 'DRIVERS', 'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL', 'RACE', 'ETHNICITY', 
                'GENDER_x', 'BIRTHPLACE', 'ADDRESS_x', 'CITY_x', 'STATE_x', 'COUNTY', 'ZIP_x', 'LAT_x', 'LON_x', 'HEALTHCARE_EXPENSES',
                'HEALTHCARE_COVERAGE', 'Id_prov', 'ORGANIZATION_y', 'NAME_x', 'GENDER_y', 'SPECIALITY', 'ADDRESS_y', 'CITY_y', 
                'STATE_y', 'ZIP_y', 'LAT_y', 'LON_y', 'UTILIZATION_x', 'Id_careplan', 'START_careplan', 'STOP_careplan', 
                'PATIENT_careplan', 'ENCOUNTER_careplan', 'CODE_careplan', 'DESCRIPTION_careplan', 'REASONCODE_y', 
                'REASONDESCRIPTION_y', 'Id_x', 'PATIENTID', 'PROVIDERID', 'PRIMARYPATIENTINSURANCEID', 'SECONDARYPATIENTINSURANCEID',
                'DEPARTMENTID', 'PATIENTDEPARTMENTID', 'DIAGNOSIS1', 'DIAGNOSIS2', 'DIAGNOSIS3', 'DIAGNOSIS4', 'DIAGNOSIS5',
                'DIAGNOSIS6', 'DIAGNOSIS7', 'DIAGNOSIS8', 'REFERRINGPROVIDERID', 'APPOINTMENTID', 'CURRENTILLNESSDATE', 'SERVICEDATE',
                'SUPERVISINGPROVIDERID', 'STATUS1', 'STATUS2', 'STATUSP', 'OUTSTANDING1', 'OUTSTANDING2', 'OUTSTANDINGP', 
                'LASTBILLEDDATE1', 'LASTBILLEDDATE2', 'LASTBILLEDDATEP', 'HEALTHCARECLAIMTYPEID1', 'HEALTHCARECLAIMTYPEID2',
                'DATE_x', 'PATIENT_immunization', 'ENCOUNTER_x', 'CODE_immunization', 'DESCRIPTION_immunization', 'BASE_COST_x', 
                'START_y', 'STOP_y', 'PATIENT_med', 'PAYER_y', 'ENCOUNTER_y', 'CODE_x', 'DESCRIPTION_x', 'BASE_COST_y',
                'PAYER_COVERAGE_y', 'DISPENSES', 'TOTALCOST', 'REASONCODE', 'REASONDESCRIPTION', 'Id_org', 'NAME_org', 'ADDRESS_org',
                'CITY_org', 'STATE', 'ZIP_org', 'LAT', 'LON', 'PHONE_x', 'REVENUE_x', 'UTILIZATION_y', 'PATIENT_pytran', 'MEMBERID',
                'START_YEAR', 'END_YEAR', 'PAYER', 'SECONDARY_PAYER', 'OWNERSHIP', 'OWNERNAME', 'Id', 'NAME', 'ADDRESS_payer',
                'CITY_payer', 'STATE_HEADQUARTERED', 'ZIP_payer', 'PHONE_y', 'AMOUNT_COVERED', 'AMOUNT_UNCOVERED', 'REVENUE_y', 
                'COVERED_ENCOUNTERS', 'UNCOVERED_ENCOUNTERS', 'COVERED_MEDICATIONS', 'UNCOVERED_MEDICATIONS', 'COVERED_PROCEDURES',
                'UNCOVERED_PROCEDURES', 'COVERED_IMMUNIZATIONS', 'UNCOVERED_IMMUNIZATIONS', 'UNIQUE_CUSTOMERS', 'QOLS_AVG', 
                'MEMBER_MONTHS', 'DATE_y', 'PATIENT', 'ENCOUNTER', 'CATEGORY_y', 'CODE_y', 'DESCRIPTION_y', 'VALUE', 'UNITS', 'TYPE_y']
enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_obsr.set_axis(columns_name, axis=1, inplace=True)
"""

In [ ]:
#print(len(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_obsr['Id_pat'].unique())," | ",enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_obsr.shape)

In [ ]:
#It is making very big data frame
#proc = procedure.head(1650)
#enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_proc = pd.merge(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer, proc, left_on="Id_pat",right_on="PATIENT")
#print(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_proc.columns)

In [ ]:
#this merging is giving zero record because there is no primary and foreign key is matching, so we will merge this later

#before it was not adding any new record to the dataset but now it is even all the records
#enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_dev = pd.merge(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer,devices, left_on="Id_pat",right_on="PATIENT")
#print(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_dev.columns)

In [ ]:
#print(len(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_dev['Id_pat'].unique())," | ",enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_dev.shape)

In [ ]:
#Whole dataset is not attaching it is removing all the records
#imstu = imaging_studies.head(20000)
#enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_imstu = pd.merge(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer, imaging_studies, left_on="PAYER",right_on="Id")
#print(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_imstu.columns)

In [ ]:
#df_columns = enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_imstu.columns.values.tolist()
#print(df_columns)
#column_dup = [k for k,v in Counter(df_columns).items() if v > 1]
#print(column_dup)

In [ ]:
#print(len(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_imstu['Id_pat'].unique())," | ",enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_imstu.shape)

In [ ]:
#I have tries with 10000 then 9000 then  5000 then 2000 then 1000 then 800 then 500 then 400... 
#...then 300 then 150: and 150 is giving no record and 200 is giving so much record it even exceeds the RAM limit
#supp = supplies.head(200)
#enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_supp = pd.merge(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer, supp, left_on="Id_pat",right_on="PATIENT")
#print(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_supp.columns)

In [ ]:
#print(len(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_supp['Id_pat'].unique())," | ",enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_supp.shape)

In [ ]:
#clmtrans = claim_transaction.head(12500)
#enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_clmtrans = pd.merge(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer, clmtrans, 
#                                                                              left_on="Id_pat",right_on="PATIENTID")
#print(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_clmtrans.columns)

In [ ]:
#print(len(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_clmtrans['Id_pat'].unique())," | ",enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_clmtrans.shape)

In [ ]:
#Here in this cell I am trying to merge just 900 rows of condition dataset
#right now RAM usage is 6.8GB
#now taking only 800 rows

#now lets increase the size to 900, please see RAM
#it iwill creash the session, session is crashed because only a 100 more rows are creating 
#so many possiblities that it is exceeding RAM limit

#HENCE THERE IS NO WAY TO MERGE SUCH DATASET BECAUSE IF I USE LESS IT WILL REMOVE ALLT 
#THE ROWS IN THE MERGED DATA IF I USE MORE THAN IT IS CREATING SO SO MUCH POSSIBILITIES
#THANK YOU, SIR

#cond = condition.head(800) 
#enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_cond = pd.merge(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer, cond, left_on="Id_pat",right_on="PATIENT")
#print(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_cond.columns)

In [ ]:
#lest check how much recored have merged
#as you can see it is neglecting allt he records which were alredy in the dmerged ataset
#print(len(enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_cond['Id_pat'].unique())," | ",enco_alg_pat_prov_carpln_claim_immun_med_org_pytran_payer_cond.shape)